In [1]:
import pandas as pd
import numpy as np
import csv
import statsmodels.api as sm
import os

In [2]:
# Path to the directory where all CSV files are stored
directory = "/Users/thomaslee/Desktop/G1-2/Computational/HMDA_Story"

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Check if the list is empty
if not csv_files:
    raise ValueError(f"No CSV files found in directory {directory}")

# Initialize an empty list to store DataFrames
dataframes_list = []

# Loop over the list of csv files
for file in csv_files:
    # Construct full file path
    file_path = os.path.join(directory, file)
    # Read the csv file and append it to the list of DataFrames
    try:
        dataframes_list.append(pd.read_csv(file_path))
    except Exception as e:
        print(f"Failed to read {file_path} due to {e}")

# Check if any dataframes were added to the list
if not dataframes_list:
    raise ValueError("No dataframes were created. Check the CSV files and their readability.")

# Combine all DataFrames into one
combined_df = pd.concat(dataframes_list, ignore_index=True)

# Draw a 1/5 random sample from the combined DataFrame
sample_df = combined_df.sample(frac=0.2, random_state=1)

/var/folders/1r/2wpzk2cd1ln3_nc94hvh0cqm0000gn/T/ipykernel_37478/154030911.py:20: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes_list.append(pd.read_csv(file_path))
/var/folders/1r/2wpzk2cd1ln3_nc94hvh0cqm0000gn/T/ipykernel_37478/154030911.py:20: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes_list.append(pd.read_csv(file_path))
/var/folders/1r/2wpzk2cd1ln3_nc94hvh0cqm0000gn/T/ipykernel_37478/154030911.py:20: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes_list.append(pd.read_csv(file_path))
/var/folders/1r/2wpzk2cd1ln3_nc94hvh0cqm0000gn/T/ipykernel_37478/154030911.py:20: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option 

In [3]:
# Rename sample_df as df
df = sample_df

In [4]:
# How many rows in df
print(len(df))

3172888


In [5]:
# Display the first few rows of the DataFrame
print(df.head())

          activity_year                   lei  derived_msa-md state_code  \
14916144           2022  549300VORTI31GZTJL53           41700         TX   
5593344            2022  549300GSCUJKJINRJ980           23060         IN   
799623             2022  549300DD5QQUHO6PCH70           22180         NC   
7590619            2022  549300DD5QQUHO6PCH70           41540         MD   
2101087            2022  549300AG64NHILB7ZP05           27260         FL   

          county_code  census_tract conforming_loan_limit  \
14916144      48259.0  4.825997e+10                     C   
5593344       18003.0  1.800300e+10                    NC   
799623        37051.0  3.705100e+10                     C   
7590619       24045.0  2.404501e+10                     C   
2101087       12089.0  1.208905e+10                     C   

         derived_loan_product_type             derived_dwelling_category  \
14916144   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
5593344    Conventional:

In [6]:
# Keep loan_purpose as 1 only
df = df[df['loan_purpose'] == 1]

In [7]:
# Keep rows where action_taken is 1, 2 or 3
df = df[df['action_taken'].isin([1, 2, 3])]

# Turn 1 and 2 into 1, and 3 into 0
df['action_taken'] = df['action_taken'].replace(2, 1)
df['action_taken'] = df['action_taken'].replace(3, 0)

In [8]:
# Drop ageapplicant that is missing or 8888
df = df[df['applicant_age'] != '8888']
df = df.dropna(subset=['applicant_age'])

In [9]:
def age_range_to_ordinal(age_range):
    mapping = {
        '<25': 0,
        '25-34': 1,
        '35-44': 2,
        '45-54': 3,
        '55-64': 4,
        '65-74': 5,
        '>74': 6
    }
    return mapping.get(age_range, None)  # Returns None if age_range is not in the mapping

In [10]:
df['applicant_age'] = df['applicant_age'].apply(age_range_to_ordinal)

In [11]:
# Keep only rows where applicant_sex is 1 or 2
df = df[df['applicant_sex'].isin([1, 2])]

In [12]:
# Keep only rows where co-applicant_sex is 1 or 2
df = df[df['co-applicant_sex'].isin([1, 2])]

In [13]:
# Add a new column named gay
# If applicant_sex and co-applicant_sex are both 1
# Otherwise mark as 0

df['gay'] = (df['applicant_sex'] == 1) & (df['co-applicant_sex'] == 1)
df['gay'] = df['gay'].astype(int)

In [14]:
# Add a new column named lesbian
# If applicant_sex and co-applicant_sex are both 2
# Otherwise mark as 0

df['les'] = (df['applicant_sex'] == 2) & (df['co-applicant_sex'] == 2)
df['les'] = df['les'].astype(int)

In [15]:
# Add a new column named women

df['women'] = (df['applicant_sex'] == 2) & (df['co-applicant_sex'] == 1)
df['women'] = df['women'].astype(int)

In [16]:
df = df[df['applicant_race-1'] != df['co-applicant_race-1']]

In [17]:
# Find out Black

df['black'] = (df['applicant_race-1'] == 3) & (df['co-applicant_race-1'] == 3)
df['black'] = df['black'].astype(int)

In [18]:
# Find out Hispanic

df['hispanic'] = (df['applicant_ethnicity-1'] == 1) & (df['co-applicant_ethnicity-1'] == 1)
df['hispanic'] = df['hispanic'].astype(int)

In [19]:
# Find out Asian

df['asian'] = (df['applicant_race-1'] == 2) & (df['co-applicant_race-1'] == 2)
df['asian'] = df['asian'].astype(int)

In [20]:
# Filter out na values in debt_to_income_ratio
df = df.dropna(subset=['debt_to_income_ratio'])

# Filter out debt_to_income_ratio that is Exempt and drop it
df = df[df['debt_to_income_ratio'] != 'Exempt']

In [21]:
# Check if there are NA values in debt_to_income_ratio
print(df['debt_to_income_ratio'].isna().sum())

0


In [22]:
# Mapping from ratio to ordinal
ordinal_mapping = {
    '<20%': 0,
    '20%-<30%': 1,
    '30%-<36%': 2,
    '36':3,
    '37':4,
    '38':5,
    '39':6,
    '40':7,
    '41':8,
    '42':9,
    '43':10,
    '44':11,
    '45':12,
    '46':13,
    '47':14,
    '48':15,
    '49':16,
    '50%-60%':17,
    '>60%':18,
}

# Convert ratios to ordinals
def ratio_to_ordinal(ratio):
    # If the ratio is a specific percentage and not in mapping, return its corresponding ordinal
    return ordinal_mapping.get(ratio, None)

In [23]:
df['debt_to_income_ratio_new'] = df['debt_to_income_ratio'].apply(ratio_to_ordinal)

In [44]:
# Check the income column data type
print(df['income'].dtype)

# Drop rows where income is NA
df = df.dropna(subset=['income'])

# Drop all negative values in income
df = df[df['income'] > 0]

# Check if there are NA values in income
print(df['income'].isna().sum())

# Convert income to log income
df['log_income'] = np.log(df['income'])

float64
0


In [45]:
# Check number of NA values in log_income
print(df['log_income'].isna().sum())

0


In [25]:
print(df['property_value'].dtype)

# Drop rows where income is NA
df = df.dropna(subset=['property_value'])

# Drop rows where income is Exempt
df = df[df['property_value'] != 'Exempt']

# Convert property_value to float
df['property_value'] = df['property_value'].astype(float)

object


In [49]:
# Keep only rows where property_value is positive
df = df[df['property_value'] > 0]

# Convert property_value to log property_value
df['log_property_value'] = np.log(df['property_value'])

In [26]:
# Convert loan_to_value_ratio to float
df['loan_to_value_ratio'] = df['loan_to_value_ratio'].str.replace('%', '').astype(float)

# Drop rows where loan_to_value_ratio is NA
df = df.dropna(subset=['loan_to_value_ratio'])

In [27]:
# Replace 'NA', 'na', 'N/A', 'Exempt', etc., with numpy.nan
df['loan_to_value_ratio'] = df['loan_to_value_ratio'].replace(['NA', 'na', 'N/A', 'Exempt', '', ' '], np.nan)

# Drop rows with missing values in 'loan_to_value_ratio'
df = df.dropna(subset=['loan_to_value_ratio'])

In [28]:
# How many rows in df
print(len(df))

23222


In [50]:
# Define your independent variables X (add your control variables as needed)
X = df[['gay','les','women','debt_to_income_ratio_new','log_income','log_property_value','loan_to_value_ratio','applicant_age']]

# Add a constant to the independent variables
X = sm.add_constant(X)

# Define your dependent variable y
y = df['action_taken']

In [51]:
# Fit the logistic regression model
model = sm.Logit(y, X)
result = model.fit()

# Print the summary of the regression
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.269941
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           action_taken   No. Observations:                23136
Model:                          Logit   Df Residuals:                    23127
Method:                           MLE   Df Model:                            8
Date:                Tue, 02 Apr 2024   Pseudo R-squ.:                 0.07356
Time:                        19:28:03   Log-Likelihood:                -6245.3
converged:                       True   LL-Null:                       -6741.2
Covariance Type:            nonrobust   LLR p-value:                9.213e-209
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -4.4675      0.620     -7.206      0.000      -5.683